In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith(("1.13.", "2.0.")) and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 793.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61406 sha256=d9c2ba33609db1e021fa21a3af72a5c9ff91caa535aa40134bb7ff93100bc6e4
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31530 sha256=3c303df152f8feca688556b6abe51f4f8af7f840f1c68f6ff498a48ea03c06b7
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath
Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py310_cu118_pyt201/download.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 179.2 MB/s eta 0:00:00


In [4]:
import torch
from pytorch3d.io import load_obj, save_obj, load_objs_as_meshes
from pytorch3d.structures import Meshes
from pytorch3d.utils import ico_sphere
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import (
    chamfer_distance,
    mesh_edge_loss,
    mesh_laplacian_smoothing,
    mesh_normal_consistency,
)

from pytorch3d.vis.texture_vis import texturesuv_image_matplotlib

from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras,
    PointLights,
    RasterizationSettings,
    MeshRenderer,
    SoftPhongShader,
    MeshRasterizer,
    TexturesUV,
    TexturesAtlas
)
from torchvision.utils import make_grid

import numpy as np

from tqdm.notebook import tqdm

%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib as mpl
# mpl.rcParams['savefig.dpi']=80
# mpl.rcParams['figure.dpi']=80

if torch.cuda.is_available():
  device=torch.device("cuda:0")
else:
  device=torch.device('cpu')
  print('cannot find gpu')



In [5]:
!pwd

/content


In [6]:
!ls

drive  sample_data


In [7]:
import os
os.chdir('drive/MyDrive/Colab Notebooks')

In [8]:
!ls

 1_b.gif
 2.1.gif
 3_b.gif
 camera_position_optimization.ipynb
 data
 deform_form.ipynb
 plot_image_grid.py
 plotly_rendered_meshes.ipynb
 q_5-1_pc1.gif
 q_5-1_pc2.gif
 q_5-1_pc_union.gif
 q_5-2.gif
 q_5-3.gif
 rendering_generic_3d_representations.ipynb
 rendering_parametric_and_implicit.ipynb
 rendering_point_clouds_from_rgbd.ipynb
 rendering_texture_obj.ipynb
 render_tetrahedron_and_retexturing.ipynb
 render_texture_mesh_and_dolly_zoom.ipynb
'sample_pointcloud_from_mesh(problem).ipynb'


In [10]:
!pip install matplotlib

In [11]:
import os
import torch
import numpy as np
from tqdm.notebook import tqdm
import imageio
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from skimage import img_as_ubyte

from skimage import img_as_ubyte

# io utils
from pytorch3d.io import load_obj

# datastructures
from pytorch3d.structures import Meshes

# 3D transformations functions
from pytorch3d.transforms import Rotate, Translate

# rendering components
from pytorch3d.renderer import (
    FoVPerspectiveCameras, look_at_view_transform, look_at_rotation,
    RasterizationSettings, MeshRenderer, MeshRasterizer, BlendParams,
    SoftSilhouetteShader, HardPhongShader, PointLights, TexturesVertex,
)

In [12]:
!mkdir -p data
!wget -P data https://dl.fbaipublicfiles.com/pytorch3d/data/teapot/teapot.obj

--2023-08-10 17:18:28--  https://dl.fbaipublicfiles.com/pytorch3d/data/teapot/teapot.obj
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 52.84.251.15, 52.84.251.106, 52.84.251.27, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|52.84.251.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 152595 (149K) [text/plain]
Saving to: ‘data/teapot.obj’

teapot.obj          100%[===================>] 149.02K  --.-KB/s    in 0.005s  

2023-08-10 17:18:28 (32.3 MB/s) - ‘data/teapot.obj’ saved [152595/152595]



In [13]:
!pwd

/content/drive/MyDrive/Colab Notebooks


In [27]:
if torch.cuda.is_available():
  device=torch.device("cuda:0")
else:
  device=torch.device("cpu")
  print("nto using gpu")


verts, faces, aux=load_obj("./data/teapot.obj")

faces=faces.verts_idx

verts_rgb=torch.ones_like(verts)
print(verts_rgb.shape)

textures=TexturesVertex(verts_features=[verts_rgb.to(device)])

teapot_mesh=Meshes(verts=[verts.to(device)], faces=[faces.to(device)], textures=textures)



torch.Size([1292, 3])


/usr/local/lib/python3.10/dist-packages/pytorch3d/io/obj_io.py:544: UserWarning:

No mtl file provided



In [23]:
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene

In [28]:
fig = plot_scene({
    "subplot1": {
        "teapot_mesh": teapot_mesh
    }
})
fig.show()

In [49]:
def mesh_render(image_size=512, lights=None, device=device, num_view=None):
  raster_settings=RasterizationSettings(
      image_size=image_size,
      blur_radius=0.0,
      faces_per_pixel=1
  )

  lights=PointLights(device=device, location=[[0,0,-1]])



  #dist, elevation, azim
  azim=torch.linspace(0,360, num_view)
  R,T=look_at_view_transform(5, azim, 0)
  cameras=FoVPerspectiveCameras(R=R, T=T, device=device)

  renderer=MeshRenderer(
      rasterizer=MeshRasterizer(raster_settings=raster_settings, cameras=cameras),
      shader=HardPhongShader(device=device, cameras=cameras, lights=lights)
  )

  return renderer


In [51]:
import imageio

num_view=12

teapot_meshes=teapot_mesh.extend(num_view)


renderer=mesh_render(num_view=num_view)
images=renderer(teapot_meshes)
print(images.shape)
cpu_image=[]

for i in range(len(images)):
  cpu_image.append(images[i, ... , :3].cpu().numpy())

print(cpu_image[0].shape)
#we plot it with a gif
imageio.mimsave('./tea_pot.gif', cpu_image, fps=10)

#plot with an image
# cpu_image=image[0, ..., :3].cpu().numpy()

# plt.figure(figsize=(7,7))
# plt.imshow(cpu_image)
# plt.axis("off")

#


torch.Size([12, 512, 512, 4])
(512, 512, 3)
